In [ ]:
# Combine DataFrames
#df_a is dropdown data, df_b map data
df_combined = pd.concat([df_a, df_b]).assign(combined_key = lambda file: file.apply(lambda x: "_".join(
    [x['district_id'] if pd.notna(x['district_id']) else "NONE" ,
    x['tehsil'] if pd.notna(x['tehsil']) else "NONE",
    x['village'] if pd.notna(x['village']) else "NONE",
    x['village_id'] if pd.notna(x['village_id']) else "NONE"]
),axis=1))

# Method 1

In [ ]:
count_series = df_combined['combined_key'].value_counts()

# Identify keys that appear in multiple tehsils
duplicate_keys = count_series[count_series > 1].index.tolist()

# Filter the DataFrame to keep only relevant entries
df_duplicates = df_combined[df_combined['combined_key'].isin(duplicate_keys)]

# Sort and drop duplicates based on priority -- here dropdown most priority
df_final = df_combined.drop_duplicates(subset=['combined_key'], keep='first')

df_final

# Method 2

In [ ]:
df_merge = df_combined.merge(df_combined.groupby(by='combined_key',as_index=False).agg({'tehsil_id':list}),
                  how='left',
                  on='combined_key').drop_duplicates(subset='combined_key').assign(**{'tehsil_id_x':lambda file: file.apply(lambda x: [i for i in x['tehsil_id_y'] if bool(re.search(r'^ctl00_ContentPlaceHolder1',i))][0] if 
                                                                                                                                      len(x['tehsil_id_y'])>1 else x['tehsil_id_y'][0],axis=1)})


# Drop combined_key and rename columns
df_final = df_merge.drop(columns=['combined_key']).rename(
    columns={
        'tehsil_id_x': 'tehsil_id_dropdown',
        'tehsil_id_y': 'tehsil_id_map'
    }
)



In [ ]:
df_final.to_csv('RJ Master Sheet.csv', index=False, encoding='utf-8')